In [50]:
# import dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import requests

In [51]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [52]:
# URL of page to be scraped
url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"

In [53]:
browser.visit(url)

In [54]:
#creating a Beautiful Soup object 
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [55]:
# results = soup.find("div", class_="content_title").get_text(strip=True)
# results

'Mars Now'

In [61]:
results = soup.find("div", class_="list_text")

<div class="list_text"><div class="list_date">November 18, 2020</div><div class="content_title"><a href="/news/8801/hear-audio-from-nasas-perseverance-as-it-travels-through-deep-space/" target="_self">Hear Audio From NASA's Perseverance As It Travels Through Deep Space</a></div><div class="article_teaser_body">The first to be rigged with microphones, the agency's latest Mars rover picked up the subtle sounds of its own inner workings during interplanetary flight.</div></div>

In [68]:
news_title = results.find("div", class_="content_title").text
news_title

"Hear Audio From NASA's Perseverance As It Travels Through Deep Space"

In [66]:
news_p = results.find("div", class_="article_teaser_body").text
news_p

"The first to be rigged with microphones, the agency's latest Mars rover picked up the subtle sounds of its own inner workings during interplanetary flight."

In [69]:
# slide = soup.select_one("ul.item_list")
# slide

In [70]:
# slide_list = slide.find_all("li", class_="slide")
# slide_list

In [46]:
# teaser = soup.find("div", class_="article_teaser_body").get_text()
# teaser

"The first to be rigged with microphones, the agency's latest Mars rover picked up the subtle sounds of its own inner workings during interplanetary flight."